<a href="https://colab.research.google.com/github/Srija-Mukherjee-31/Fake-News-Detector/blob/main/Fake_News_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install numpy pandas scikit-learn nltk transformers torch




In [ ]:
import pandas as pd
import numpy as np
import re
import nltk
import gradio as gr

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

from transformers import pipeline

nltk.download('stopwords')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
fake = pd.read_csv("/FAKE.csv")
true = pd.read_csv("/TRUE.csv")

fake["label"] = 0   # fake
true["label"] = 1   # real

In [ ]:

data = pd.concat([fake, true], axis=0)

# reduce size for faster execution
data = data.sample(10000, random_state=42).reset_index(drop=True)

Text preprocessing

In [ ]:
def clean_text(text):
    text = text.lower()
    text = re.sub(r'\[.*?\]', '', text)
    text = re.sub(r'[^a-zA-Z ]', ' ', text)
    text = re.sub(r'\s+', ' ', text)
    return text

data["text"] = data["text"].apply(clean_text)

Train-test split

In [ ]:
X = data["text"]
y = data["label"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, random_state=42
)


Feature extraction

In [ ]:
vectorizer = TfidfVectorizer(stop_words="english", max_df=0.7)
X_train = vectorizer.fit_transform(X_train)
X_test = vectorizer.transform(X_test)

Model training

In [ ]:
model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)

LogisticRegression(max_iter=1000)

Model evaluation

In [ ]:
y_pred = model.predict(X_test)
print("Model Accuracy:", accuracy_score(y_test, y_pred))

Accuracy: 0.9676


In [ ]:
summarizer = pipeline(
    "summarization",
    model="sshleifer/distilbart-cnn-12-6"
)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.22G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.22G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

Device set to use cpu


In [ ]:
def fake_news_detector(news_text):
    cleaned = clean_text(news_text)
    vector = vectorizer.transform([cleaned])

    prediction = model.predict(vector)[0]
    probability = model.predict_proba(vector)[0][1] * 100

    label = "Likely Real News" if prediction == 1 else "Possibly Fake News"
    credibility = round(probability, 2)

    if len(news_text.split()) < 80:
        summary = "Article too short for summarization."
    else:
        summary = summarizer(
            news_text,
            max_length=120,
            min_length=40,
            do_sample=False
        )[0]["summary_text"]

    return label, credibility, summary



In [ ]:
interface = gr.Interface(
    fn=fake_news_detector,
    inputs=gr.Textbox(
        lines=10,
        placeholder="Paste news article here...",
        label="News Article"
    ),
    outputs=[
        gr.Textbox(label="Prediction"),
        gr.Textbox(label="Credibility Score (%)"),
        gr.Textbox(label="Summary")
    ],
    title="Fake News Detector for Students",
    description="An AI-based system to detect fake news, assess credibility, and generate summaries."
)

interface.launch()


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://2b13dbd59a4f096570.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
